# 国土地理院の市区町村データ

http://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N03-v2_3.html
ここでzipをDLできる。

ファイルのURLは
http://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2019/N03-190101_21_GML.zip
など（21_GMLの部分が、01_GML〜47_GMLまで）

In [23]:
import pandas as pd

In [6]:
import geojson as gj
file = "N03-19_13_190101.geojson"
obj = gj.load(open(file))

In [7]:
type(obj)

geojson.feature.FeatureCollection

FeatureCollectionとして提供されている。最初の方の要素を見てみる

In [14]:
print(open(file).read()[:500])

{
"type": "FeatureCollection",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::6668" } },
"features": [
{ "type": "Feature", "properties": { "N03_001": "東京都", "N03_002": null, "N03_003": null, "N03_004": "千代田区", "N03_007": "13101" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 139.77286583728858, 35.703702135445269 ], [ 139.77279358035946, 35.703120197639066 ], [ 139.77286608280349, 35.703113531864062 ], [ 139.77290635534405, 35.703109639598274 ], [ 139.77329691


propertiesとgeometryを見ていけば良さそう。

In [37]:
fts = obj["features"]
names_idx = [f"N03_00{i}" for i in [1,2,3,4,7]]
names = [[ft["properties"][n] for n in names_idx] for ft in fts]
n_polys = [ len(ft["geometry"]["coordinates"]) for ft in fts]
coordinates = [ ft["geometry"]["coordinates"] for ft in fts ] 

In [38]:
df = pd.DataFrame(names)
df.columns = names_idx
df["n_polys"] = n_polys
df["coordinate"] = coordinates

In [39]:
df.groupby("n_polys").count()

,N03_001,N03_002,N03_003,N03_004,N03_007,coordinate
n_polys,,,,,,
1,6184,0,5922,6184,6060,6184
2,3,0,0,3,3,3


飛び地になってるのが3個いる

In [40]:
import folium

In [48]:
tobichi = df.loc[df["n_polys"]>1,]
tobichi

,N03_001,N03_002,N03_003,N03_004,N03_007,n_polys,coordinate
122,東京都,None,None,町田市,13209,2,"[[[139.338483, 35.622969], [139.338555, 35.622..."
123,東京都,None,None,小金井市,13210,2,"[[[139.525426, 35.712921], [139.526263, 35.712..."
134,東京都,None,None,武蔵村山市,13223,2,"[[[139.37091, 35.771434], [139.371174, 35.7713..."


In [60]:
def swap(p):
    return [p[1], p[0]]

fmap = folium.Map(
    location = swap(tobichi["coordinate"].iloc[0][0][0])
)
for i in tobichi.index:
    cds = tobichi["coordinate"][i]
    for j, cd in enumerate(cds):
        poly = folium.GeoJson(
            dict(
                type = "Polygon",
                coordinates = [cd],
                pos = j
            ),
            style_function= lambda f: dict(
                color = "red" if f["geometry"]["pos"] > 0 else "blue"
            ))
        poly.add_to(fmap)
fmap

In [62]:
fmap2 = folium.Map(
    location = swap(tobichi["coordinate"].iloc[0][0][0])
)
for i in tobichi.index:
    poly = folium.GeoJson(
        dict(
            type = "Polygon",
            coordinates = tobichi["coordinate"][i],
            pos = j
        ))
    poly.add_to(fmap2)
fmap2

飛び地は無視して1個目のポリゴンだけ着目しとけば良さそう。

In [66]:
df.head(20)

,N03_001,N03_002,N03_003,N03_004,N03_007,n_polys,coordinate
0,東京都,None,None,千代田区,13101,1,"[[[139.772866, 35.703702], [139.772794, 35.703..."
1,東京都,None,None,中央区,13102,1,"[[[139.783406, 35.696449], [139.784588, 35.696..."
2,東京都,None,None,港区,13103,1,"[[[139.771289, 35.628412], [139.771276, 35.628..."
3,東京都,None,None,港区,13103,1,"[[[139.766891, 35.627742], [139.767177, 35.627..."
4,東京都,None,None,港区,13103,1,"[[[139.770216, 35.631988], [139.770462, 35.631..."
5,東京都,None,None,港区,13103,1,"[[[139.758007, 35.633798], [139.75801, 35.6336..."
6,東京都,None,None,港区,13103,1,"[[[139.758522, 35.633802], [139.758521, 35.633..."
7,東京都,None,None,港区,13103,1,"[[[139.766667, 35.63298], [139.76654, 35.63293..."
8,東京都,None,None,港区,13103,1,"[[[139.77659, 35.636113], [139.776615, 35.6360..."
9,東京都,None,None,港区,13103,1,"[[[139.756209, 35.636886], [139.756196, 35.636..."


港区が複数レコードある

In [68]:
df.tail(20)

,N03_001,N03_002,N03_003,N03_004,N03_007,n_polys,coordinate
6167,東京都,None,None,所属未定地,None,1,"[[[139.917625, 31.887863], [139.917582, 31.887..."
6168,東京都,None,None,所属未定地,None,1,"[[[139.918951, 31.887923], [139.918933, 31.887..."
6169,東京都,None,None,所属未定地,None,1,"[[[139.918922, 31.88807], [139.918901, 31.8880..."
6170,東京都,None,None,所属未定地,None,1,"[[[139.918014, 31.888168], [139.917999, 31.888..."
6171,東京都,None,None,所属未定地,None,1,"[[[139.917973, 31.888334], [139.91796, 31.8883..."
6172,東京都,None,None,所属未定地,None,1,"[[[139.918423, 31.888541], [139.918428, 31.888..."
6173,東京都,None,None,所属未定地,None,1,"[[[139.918598, 31.888539], [139.918569, 31.888..."
6174,東京都,None,None,所属未定地,None,1,"[[[139.918553, 31.888686], [139.91859, 31.8886..."
6175,東京都,None,None,所属未定地,None,1,"[[[139.918645, 31.888862], [139.918667, 31.888..."
6176,東京都,None,None,所属未定地,None,1,"[[[140.052502, 31.43806], [140.052463, 31.4380..."


所属未定地というのもいる